<a href="https://colab.research.google.com/github/Sidhved/Machine-Learning/blob/main/ClassificationBank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#path for dataset
path = "/content/drive/MyDrive/dataSets/banking.csv"

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
data = pd.read_csv(path)
data.fillna(data.mean(), inplace = True)
print(data.shape)
print(list(data.columns))

(41188, 21)
['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y']


In [4]:
data.info

<bound method DataFrame.info of        age          job  marital  ... euribor3m nr_employed  y
0       44  blue-collar  married  ...     4.963      5228.1  0
1       53   technician  married  ...     4.021      5195.8  0
2       28   management   single  ...     0.729      4991.6  1
3       39     services  married  ...     1.405      5099.1  0
4       55      retired  married  ...     0.869      5076.2  1
...    ...          ...      ...  ...       ...         ... ..
41183   59      retired  married  ...     4.866      5228.1  0
41184   31    housemaid  married  ...     4.860      5191.0  0
41185   42       admin.   single  ...     4.857      5191.0  0
41186   48   technician  married  ...     0.742      5017.5  0
41187   25      student   single  ...     4.859      5191.0  0

[41188 rows x 21 columns]>

In [5]:
#grouping basic.9y, basic.6y and basic.4y
data['education']=np.where(data['education'] =='basic.9y', 'Basic', data['education'])
data['education']=np.where(data['education'] =='basic.6y', 'Basic', data['education'])
data['education']=np.where(data['education'] =='basic.4y', 'Basic', data['education'])
data['education'].unique()

array(['Basic', 'unknown', 'university.degree', 'high.school',
       'professional.course', 'illiterate'], dtype=object)

In [6]:
count_no_sub = len(data[data['y']==0])
count_sub = len(data[data['y']==1])
pct_of_no_sub = count_no_sub/(count_no_sub+count_sub)
print("percentage of no subscription is", pct_of_no_sub*100)
pct_of_sub = count_sub/(count_no_sub+count_sub)
print("percentage of subscription", pct_of_sub*100)

percentage of no subscription is 88.73458288821988
percentage of subscription 11.265417111780131


In [7]:
print(data.groupby('job').mean())
print(data.groupby('marital').mean())
print(data.groupby('education').mean())

                     age    duration  ...  nr_employed         y
job                                   ...                       
admin.         38.187296  254.312128  ...  5164.125350  0.129726
blue-collar    39.555760  264.542360  ...  5175.615150  0.068943
entrepreneur   41.723214  263.267857  ...  5176.313530  0.085165
housemaid      45.500000  250.454717  ...  5179.529623  0.100000
management     42.362859  257.058140  ...  5166.650513  0.112175
retired        62.027326  273.712209  ...  5122.262151  0.252326
self-employed  39.949331  264.142153  ...  5170.674384  0.104856
services       37.926430  258.398085  ...  5171.600126  0.081381
student        25.894857  283.683429  ...  5085.939086  0.314286
technician     38.507638  250.232241  ...  5175.648391  0.108260
unemployed     39.733728  249.451677  ...  5157.156509  0.142012
unknown        45.563636  239.675758  ...  5172.931818  0.112121

[12 rows x 11 columns]
                age    duration  campaign  ...  euribor3m  nr_empl

In [8]:
print(data['education'].value_counts())

Basic                  12513
university.degree      12168
high.school             9515
professional.course     5243
unknown                 1731
illiterate                18
Name: education, dtype: int64


In [9]:
#convert educational data into categorical equivalent
replace_edu = {'education': {'Basic': 1, 'university.degree': 2, 'high.school': 3, 'professional.course': 4, 'unknown': 5, 'illiterate': 6}}
dataEdu = data.copy()
data = dataEdu.replace(replace_edu)
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,1,unknown,yes,no,cellular,aug,thu,210,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,5,no,no,no,cellular,nov,fri,138,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,2,no,yes,no,cellular,jun,thu,339,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,3,no,no,no,cellular,apr,fri,185,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,1,no,yes,no,cellular,aug,fri,137,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,59,retired,married,3,unknown,no,yes,telephone,jun,thu,222,1,999,0,nonexistent,1.4,94.465,-41.8,4.866,5228.1,0
41184,31,housemaid,married,1,unknown,no,no,telephone,may,thu,196,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,0
41185,42,admin.,single,2,unknown,yes,yes,telephone,may,wed,62,3,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
41186,48,technician,married,4,no,no,yes,telephone,oct,tue,200,2,999,0,nonexistent,-3.4,92.431,-26.9,0.742,5017.5,0


In [10]:
print(data['job'].value_counts())

admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64


In [11]:
#convert job data into categorical equivalent
replace_job = {'job': {'admin.': 1, 'blue-collar': 2, 'technician': 3, 'services': 4, 'management': 5, 'retired': 6, 'entrepreneur': 7, 'self-employed': 8, 'housemaid': 9, 'unemployed': 10, 'student': 11, 'unknown': 12}}
dataJob = data.copy()
data = dataJob.replace(replace_job)

In [12]:
print(data['month'].value_counts())

may    13769
jul     7174
aug     6178
jun     5318
nov     4101
apr     2632
oct      718
sep      570
mar      546
dec      182
Name: month, dtype: int64


In [13]:
#convert month data into categorical equivalent
replace_mon = {'month': {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}}
dataMon = data.copy()
data = dataMon.replace(replace_mon)

In [14]:
print(data['poutcome'].value_counts())

nonexistent    35563
failure         4252
success         1373
Name: poutcome, dtype: int64


In [15]:
#convert poutcome data into categorical equivalent
replace_Pout = {'poutcome': {'failure': 0, 'success': 1, 'nonexistent':2}}
dataPout = data.copy()
data = dataPout.replace(replace_Pout)

In [16]:
print(data['marital'].value_counts())

married     24928
single      11568
divorced     4612
unknown        80
Name: marital, dtype: int64


In [17]:
#convert Marital data into categorical equivalent
replace_Mar = {'marital': {'married': 1, 'single': 2, 'divorced':3, 'unknown': 4}}
dataMar = data.copy()
data = dataMar.replace(replace_Mar)

In [18]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,2,1,1,unknown,yes,no,cellular,8,thu,210,1,999,0,2,1.4,93.444,-36.1,4.963,5228.1,0
1,53,3,1,5,no,no,no,cellular,11,fri,138,1,999,0,2,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,5,2,2,no,yes,no,cellular,6,thu,339,3,6,2,1,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,4,1,3,no,no,no,cellular,4,fri,185,2,999,0,2,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,6,1,1,no,yes,no,cellular,8,fri,137,1,3,1,1,-2.9,92.201,-31.4,0.869,5076.2,1


In [19]:
#row filtering by value
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1=data.join(cat_list)
    data=data1

cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

In [20]:
data_final=data[to_keep]
data_final.columns.values

array(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y',
       'job_1', 'job_2', 'job_3', 'job_4', 'job_5', 'job_6', 'job_7',
       'job_8', 'job_9', 'job_10', 'job_11', 'job_12', 'marital_1',
       'marital_2', 'marital_3', 'marital_4', 'education_1',
       'education_2', 'education_3', 'education_4', 'education_5',
       'education_6', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular',
       'contact_telephone', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11',
       'month_12', 'day_of_week_fri', 'day_of_week_mon',
       'day_of_week_thu', 'day_of_week_tue', 'day_of_week_wed',
       'poutcome_0', 'poutcome_1', 'poutcome_2'], dtype=object)

**Over-Sampling using SMOTE**

In [21]:
X = data_final.loc[:, data_final.columns != 'y']
y = data_final.loc[:, data_final.columns == 'y']
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['y']==0]))
print("Number of subscription",len(os_data_y[os_data_y['y']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


length of oversampled data is  51134
Number of no subscription in oversampled data 25567
Number of subscription 25567
Proportion of no subscription data in oversampled data is  0.5
Proportion of subscription data in oversampled data is  0.5


**SET DATA**

In [26]:
#cloumn filtering
cols = ['euribor3m', 'job_2', 'job_9',  'marital_4', 'education_6', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_10', 'month_11', 'poutcome_0', 'poutcome_1']
X = os_data_X[cols]
y = os_data_y['y']      

**IMPLEMENTING THE MODEL**

In [27]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.557850
         Iterations 7
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.195     
Dependent Variable: y                AIC:              57080.2468
Date:               2021-03-23 05:34 BIC:              57212.8799
No. Observations:   51134            Log-Likelihood:   -28525.   
Df Model:           14               LL-Null:          -35443.   
Df Residuals:       51119            LLR p-value:      0.0000    
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     7.0000                                       
------------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
------------------------------------------------------------------
euribor3m    -0.4409    0.0074  -59.9752  0.0000  -0.4554  -0.4265
job_2        -0.2043    0.0278   -7.3497  0.0000  -0.2588  -0.

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.74
